In [ ]:
import json
from collections import defaultdict, Counter
import os
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, hamming_loss, accuracy_score

# --- 1. Hàm đọc dữ liệu ASCA JSON (Giữ nguyên) ---
def load_asca_data(filepath):
    """Đọc dữ liệu từ file JSON định dạng ASCA."""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
            if "sentences" in data and isinstance(data["sentences"], list):
                return data["sentences"]
            else:
                print(f"Lỗi: Cấu trúc JSON không đúng trong file {filepath}.")
                return []
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file {filepath}")
        return []
    except json.JSONDecodeError:
        print(f"Lỗi: File {filepath} không phải là định dạng JSON hợp lệ.")
        return []
    except Exception as e:
        print(f"Lỗi không xác định khi đọc file: {e}")
        return []

# --- 2. Hàm huấn luyện và đánh giá mô hình đa nhãn (Giữ nguyên) ---
def train_evaluate_multilabel(X_train, y_train, X_test, y_test, labels_list, task_name=""):
    """Huấn luyện và đánh giá mô hình đa nhãn (TF-IDF + OneVsRest)."""
    print(f"\n--- Bắt đầu {task_name} ---")

    # Lựa chọn base classifier
    # base_classifier = LogisticRegression(solver='liblinear', random_state=42, class_weight='balanced')
    base_classifier = LinearSVC(random_state=42, dual="auto", class_weight='balanced')

    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(min_df=3, max_df=0.9, ngram_range=(1, 2))),
        ('clf', OneVsRestClassifier(base_classifier, n_jobs=-1))
    ])

    print("  Bắt đầu huấn luyện...")
    pipeline.fit(X_train, y_train)
    print("  Huấn luyện hoàn tất.")

    print("  Bắt đầu dự đoán...")
    y_pred = pipeline.predict(X_test)
    print("  Dự đoán hoàn tất.")

    print("\n  --- Kết quả đánh giá ---")
    h_loss = hamming_loss(y_test, y_pred)
    subset_acc = accuracy_score(y_test, y_pred)
    print(f"  Hamming Loss: {h_loss:.4f}")
    print(f"  Subset Accuracy: {subset_acc:.4f}")

    print("\n  Classification Report:")
    report = classification_report(
        y_test,
        y_pred,
        labels=np.arange(len(labels_list)), # Cung cấp chỉ số của các lớp
        target_names=labels_list,          # Tên tương ứng của các lớp
        zero_division=0,
        digits=3
    )
    print(report)
    print(f"--- Kết thúc {task_name} ---")
    return pipeline # Trả về pipeline đã huấn luyện

# --- 3. Main Script ---
input_json_file = 'output_asca_format.txt'
raw_data = load_asca_data(input_json_file)

if not raw_data:
    print("Không có dữ liệu để xử lý.")
else:
    print(f"Đã đọc {len(raw_data)} câu từ file JSON.")
    X_texts = [item['text'] for item in raw_data]
    y_categories_raw = [
        list(set(cat_pol['category'] for cat_pol in item.get('aspectCategories', []) if cat_pol.get('category')))
        for item in raw_data
    ]
    mlb_category = MultiLabelBinarizer()
    y_categories_binarized = mlb_category.fit_transform(y_categories_raw)
    category_labels = mlb_category.classes_
    y_polarities_raw = [
        list(set(cat_pol['polarity'] for cat_pol in item.get('aspectCategories', []) if cat_pol.get('polarity')))
        for item in raw_data
    ]
    mlb_polarity = MultiLabelBinarizer()
    y_polarities_binarized = mlb_polarity.fit_transform(y_polarities_raw)
    polarity_labels = sorted([p for p in ['positive', 'negative', 'neutral'] if p in mlb_polarity.classes_])
    y_catpol_raw = [
        [f"{cat_pol['category']}#{cat_pol['polarity']}"
         for cat_pol in item.get('aspectCategories', []) if cat_pol.get('category') and cat_pol.get('polarity')]
        for item in raw_data
    ]
    mlb_catpol = MultiLabelBinarizer()
    y_catpol_binarized = mlb_catpol.fit_transform(y_catpol_raw)
    catpol_labels = mlb_catpol.classes_
    indices = np.arange(len(X_texts))
    X_train_texts, X_test_texts, y_train_cat, y_test_cat, y_train_pol, y_test_pol, y_train_cp, y_test_cp, train_indices, test_indices = train_test_split(
        X_texts,
        y_categories_binarized,
        y_polarities_binarized,
        y_catpol_binarized,
        indices,
        test_size=0.2,
        random_state=42
    )

    print("-" * 30)
    print(f"Tổng số mẫu: {len(X_texts)}")
    print(f"Kích thước tập huấn luyện: {len(X_train_texts)}")
    print(f"Kích thước tập kiểm thử: {len(X_test_texts)}")
    print("-" * 30)

    print("\n" + "="*20 + " Tác vụ 1: Aspect Category Detection (ACD) " + "="*20)
    print(f"Số lượng Category duy nhất: {len(category_labels)}")
    pipeline_acd = train_evaluate_multilabel(X_train_texts, y_train_cat, X_test_texts, y_test_cat, category_labels, "ACD")

    print("\n" + "="*20 + " Tác vụ 2: Sentiment Polarity Classification (SPC) " + "="*20)
    print(f"Số lượng Polarity duy nhất: {len(polarity_labels)}")
    pipeline_spc = train_evaluate_multilabel(X_train_texts, y_train_pol, X_test_texts, y_test_pol, polarity_labels, "SPC")

    print("\n" + "="*20 + " Tác vụ 3: Category#Polarity Pair Detection (ASCA) " + "="*20)
    print(f"Số lượng Cặp Category#Polarity duy nhất: {len(catpol_labels)}")
    pipeline_catpol = train_evaluate_multilabel(X_train_texts, y_train_cp, X_test_texts, y_test_cp, catpol_labels, "ASCA Pairs")


    print("\nHoàn thành tất cả các tác vụ.")

Đã đọc 15519 câu từ file JSON.
------------------------------
Tổng số mẫu: 15519
Kích thước tập huấn luyện: 12415
Kích thước tập kiểm thử: 3104
------------------------------

==================== Tác vụ 1: Aspect Category Detection (ACD) ====================
Số lượng Category duy nhất: 8

--- Bắt đầu ACD ---
  Bắt đầu huấn luyện...
  Huấn luyện hoàn tất.
  Bắt đầu dự đoán...
  Dự đoán hoàn tất.

  --- Kết quả đánh giá ---
  Hamming Loss: 0.1089
  Subset Accuracy: 0.4501

  Classification Report:
                             precision    recall  f1-score   support

         Course information      0.498     0.680     0.575       328
             General review      0.292     0.462     0.358       292
       Learning environment      0.812     0.794     0.803       180
Organization and management      0.555     0.660     0.603       244
     Support from lecturers      0.575     0.674     0.621       857
           Teaching quality      0.738     0.764     0.751      1269
        Test a

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
# Import đầy đủ các hàm metrics
from sklearn.metrics import classification_report, accuracy_score, f1_score
import re

# --- Configuration ---
FILE_PATH = 'combined_cleaned_file.txt' # Đường dẫn tới file dữ liệu của bạn
TEST_SIZE = 0.2 # Tỷ lệ dữ liệu dùng để kiểm thử (validation)
RANDOM_STATE = 42 # Để đảm bảo kết quả lặp lại được

# --- Load Data ---
try:
    df = pd.read_csv(FILE_PATH, sep='\t', encoding='utf-8')
    print("Đã tải dữ liệu thành công.")
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file tại đường dẫn '{FILE_PATH}'")
    exit()
except Exception as e:
    print(f"Lỗi khi đọc file: {e}")
    exit()

# --- Data Cleaning and Preparation ---
required_columns = ['Sentence Component', 'aspect', 'sentiment_text', 'sentiment']
df.dropna(subset=required_columns, inplace=True)
print(f"\nSố lượng dòng sau khi xóa NaN: {len(df)}")

# Loại bỏ các ký tự đặc biệt (Tùy chọn)
def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'colonsmilesmile|wzjwz\d+', '', text, flags=re.IGNORECASE)
        text = text.strip()
    return text
# df['Sentence Component'] = df['Sentence Component'].apply(clean_text)
# df['sentiment_text'] = df['sentiment_text'].apply(clean_text)

# --- Split Data into Training and Testing Sets (Using DataFrame) ---
train_df, test_df = train_test_split(
    df,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=df['aspect'] # Stratify theo aspect
)
print(f"\nSố lượng mẫu huấn luyện: {len(train_df)}")
print(f"Số lượng mẫu kiểm thử: {len(test_df)}")


# --- 1. Aspect Identification Model ---
print("\n--- Huấn luyện & Đánh giá Mô hình Nhận diện Aspect ---")
aspect_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
    ('clf', LogisticRegression(solver='liblinear', multi_class='auto', random_state=RANDOM_STATE))
])
print("Đang huấn luyện mô hình Aspect...")
aspect_pipeline.fit(train_df['Sentence Component'], train_df['aspect'])
print("Dự đoán Aspect trên tập kiểm thử (test_df)...")
predicted_aspects = aspect_pipeline.predict(test_df['Sentence Component'])
test_df['predicted_aspect'] = predicted_aspects
accuracy_aspect = accuracy_score(test_df['aspect'], test_df['predicted_aspect'])
print(f"Độ chính xác (Accuracy) riêng cho Aspect: {accuracy_aspect:.4f}")

# --- Hiển thị lại Classification Report cho Aspect ---
print("\nBáo cáo phân loại chi tiết cho Aspect:")
# Lấy danh sách các nhãn aspect duy nhất từ cả tập thực tế và dự đoán
all_aspect_labels = sorted(list(set(test_df['aspect']) | set(test_df['predicted_aspect'])))
# In báo cáo
print(classification_report(test_df['aspect'], test_df['predicted_aspect'], labels=all_aspect_labels, zero_division=0))
# --- Kết thúc phần hiển thị lại ---


# --- 2. Sentiment Classification Model ---
print("\n--- Huấn luyện & Đánh giá Mô hình Phân loại Sentiment ---")
sentiment_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
    ('clf', LogisticRegression(solver='liblinear', multi_class='auto', random_state=RANDOM_STATE))
])
print("Đang huấn luyện mô hình Sentiment...")
sentiment_pipeline.fit(train_df['sentiment_text'], train_df['sentiment'])
print("Dự đoán Sentiment trên tập kiểm thử (test_df)...")
predicted_sentiments = sentiment_pipeline.predict(test_df['sentiment_text'])
test_df['predicted_sentiment'] = predicted_sentiments
accuracy_sentiment = accuracy_score(test_df['sentiment'], test_df['predicted_sentiment'])
print(f"Độ chính xác (Accuracy) riêng cho Sentiment: {accuracy_sentiment:.4f}")

# --- Hiển thị lại Classification Report cho Sentiment ---
print("\nBáo cáo phân loại chi tiết cho Sentiment:")
# Lấy danh sách các nhãn sentiment duy nhất từ cả tập thực tế và dự đoán
all_sentiment_labels = sorted(list(set(test_df['sentiment']) | set(test_df['predicted_sentiment'])))
# In báo cáo
print(classification_report(test_df['sentiment'], test_df['predicted_sentiment'], labels=all_sentiment_labels, zero_division=0))
# --- Kết thúc phần hiển thị lại ---


# --- 3. Combine Predictions and Evaluate Pair Performance ---
print("\n--- Đánh giá hiệu suất dự đoán CẶP {Aspect: Sentiment} ---")

test_df['true_pair'] = test_df.apply(lambda row: f"{{{row['aspect']}: {row['sentiment']}}}", axis=1)
test_df['predicted_pair'] = test_df.apply(lambda row: f"{{{row['predicted_aspect']}: {row['predicted_sentiment']}}}", axis=1)

pair_accuracy = accuracy_score(test_df['true_pair'], test_df['predicted_pair'])
print(f"Độ chính xác (Accuracy) cho việc dự đoán đúng cả cặp {{Aspect: Sentiment}}: {pair_accuracy:.4f}")

pair_f1_weighted = f1_score(test_df['true_pair'], test_df['predicted_pair'], average='weighted', zero_division=0)
pair_f1_macro = f1_score(test_df['true_pair'], test_df['predicted_pair'], average='macro', zero_division=0)
print(f"F1-score (Weighted) cho việc dự đoán đúng cả cặp {{Aspect: Sentiment}}: {pair_f1_weighted:.4f}")
print(f"F1-score (Macro)   cho việc dự đoán đúng cả cặp {{Aspect: Sentiment}}: {pair_f1_macro:.4f}")

print("\nBáo cáo phân loại chi tiết cho từng cặp {Aspect: Sentiment}:")
all_pair_labels = sorted(list(set(test_df['true_pair']) | set(test_df['predicted_pair'])))
print(classification_report(test_df['true_pair'], test_df['predicted_pair'], labels=all_pair_labels, zero_division=0))


# --- 4. Display Predicted Aspect:Sentiment Pairs for Test Set (Example) ---
print("\n--- Hiển thị ví dụ Kết quả Dự đoán Cặp {Aspect: Sentiment} trên Tập Kiểm Thử ---")
print("(Hiển thị 20 mẫu đầu tiên của tập kiểm thử)")

columns_to_show = [
    'Sentence Component',
    'sentiment_text',
    'true_pair',
    'predicted_pair'
]
print(test_df[columns_to_show].head(20).to_string())

print("\n--- Kết thúc ---")

Đã tải dữ liệu thành công.

Số lượng dòng sau khi xóa NaN: 20779

Số lượng mẫu huấn luyện: 16623
Số lượng mẫu kiểm thử: 4156

--- Huấn luyện & Đánh giá Mô hình Nhận diện Aspect ---
Đang huấn luyện mô hình Aspect...


/home/duycute/.pyenv/versions/3.11.11/envs/Ai_ENV/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Dự đoán Aspect trên tập kiểm thử (test_df)...
Độ chính xác (Accuracy) riêng cho Aspect: 0.6894

Báo cáo phân loại chi tiết cho Aspect:
                             precision    recall  f1-score   support

         Course information       0.64      0.66      0.65       417
             General review       0.68      0.29      0.41       297
       Learning environment       0.89      0.83      0.86       223
Organization and management       0.65      0.60      0.62       301
     Support from lecturers       0.68      0.68      0.68      1152
           Teaching quality       0.69      0.80      0.74      1455
        Test and evaluation       0.75      0.60      0.67       137
                   Workload       0.69      0.63      0.66       174

                   accuracy                           0.69      4156
                  macro avg       0.71      0.64      0.66      4156
               weighted avg       0.69      0.69      0.68      4156


--- Huấn luyện & Đánh giá Mô hình

/home/duycute/.pyenv/versions/3.11.11/envs/Ai_ENV/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Dự đoán Sentiment trên tập kiểm thử (test_df)...
Độ chính xác (Accuracy) riêng cho Sentiment: 0.8604

Báo cáo phân loại chi tiết cho Sentiment:
              precision    recall  f1-score   support

    Negative       0.85      0.87      0.86      1198
     Neutral       0.70      0.55      0.62       606
    Positive       0.90      0.93      0.92      2352

    accuracy                           0.86      4156
   macro avg       0.81      0.79      0.80      4156
weighted avg       0.85      0.86      0.86      4156


--- Đánh giá hiệu suất dự đoán CẶP {Aspect: Sentiment} ---
Độ chính xác (Accuracy) cho việc dự đoán đúng cả cặp {Aspect: Sentiment}: 0.6037
F1-score (Weighted) cho việc dự đoán đúng cả cặp {Aspect: Sentiment}: 0.5938
F1-score (Macro)   cho việc dự đoán đúng cả cặp {Aspect: Sentiment}: 0.4823

Báo cáo phân loại chi tiết cho từng cặp {Aspect: Sentiment}:
                                         precision    recall  f1-score   support

         {Course information: Negativ